In [35]:
import os
import re
import time
import string
import polars as pl

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>Info</font></h3>
    
This notebook allows you to concatenate and filter the data necessary to train the Machine Learning model for the Recommendation System

# List all reviews parquet files related to reviews to be loaded

In [36]:
path = '../data/raw/reviews'
with os.scandir(path) as reviewFiles:
    reviewFiles = [file.name for file in reviewFiles if file.is_file()]
print(reviewFiles)

['processed_reviews_books_1.parquet', 'processed_reviews_books_10.parquet', 'processed_reviews_books_11.parquet', 'processed_reviews_books_12.parquet', 'processed_reviews_books_13.parquet', 'processed_reviews_books_3.parquet', 'processed_reviews_books_4.parquet', 'processed_reviews_books_5.parquet', 'processed_reviews_books_6.parquet', 'processed_reviews_books_7.parquet', 'processed_reviews_books_8.parquet', 'processed_reviews_books_9.parquet']


# Load and join all parquet files in the directory

In [37]:
df = pl.DataFrame()
for f in reviewFiles:
    n = str.format("../data/raw/reviews/{}", f)
    print("reading " + n)

    df_aux = pl.read_parquet(n)

    print("Adding..." + n)
    df = df.vstack(df_aux)
    n = ""


reading ../data/raw/reviews/processed_reviews_books_1.parquet
Adding...../data/raw/reviews/processed_reviews_books_1.parquet
reading ../data/raw/reviews/processed_reviews_books_10.parquet
Adding...../data/raw/reviews/processed_reviews_books_10.parquet
reading ../data/raw/reviews/processed_reviews_books_11.parquet
Adding...../data/raw/reviews/processed_reviews_books_11.parquet
reading ../data/raw/reviews/processed_reviews_books_12.parquet
Adding...../data/raw/reviews/processed_reviews_books_12.parquet
reading ../data/raw/reviews/processed_reviews_books_13.parquet
Adding...../data/raw/reviews/processed_reviews_books_13.parquet
reading ../data/raw/reviews/processed_reviews_books_3.parquet
Adding...../data/raw/reviews/processed_reviews_books_3.parquet
reading ../data/raw/reviews/processed_reviews_books_4.parquet
Adding...../data/raw/reviews/processed_reviews_books_4.parquet
reading ../data/raw/reviews/processed_reviews_books_5.parquet
Adding...../data/raw/reviews/processed_reviews_books_5.

In [38]:
df.sample(1)

asin,overall,reviewText,reviewerID,summary,verified,unixReviewTime
str,str,str,str,str,str,datetime[μs]
"""1475031904""","""5.0""","""Exciting and f…","""AYAQQG8UNG5FG""","""Fun read.""","""true""",2016-03-02 00:00:00


In [39]:
df.shape

(2620482, 7)

In [40]:
df.null_count()

asin,overall,reviewText,reviewerID,summary,verified,unixReviewTime
u32,u32,u32,u32,u32,u32,u32
0,0,1260,0,796,0,0


In [41]:
df = df.drop_nulls()

In [42]:
df.shape

(2618468, 7)

Save it as temporal parquet file

In [43]:
df.write_parquet("../data/processed/All_reviews.parquet")

In [44]:
reviews = pl.read_parquet("../data/processed/All_reviews.parquet")

Load the metadata related to books category

In [45]:
metadata = pl.read_parquet("../data/raw/all_metadata_books_in_top25k.parquet")

In [46]:
reviews.shape

(2618468, 7)

In [47]:
metadata.shape

(12049, 9)

In [48]:
reviews.sample(1)

asin,overall,reviewText,reviewerID,summary,verified,unixReviewTime
str,str,str,str,str,str,datetime[μs]
"""0385349173""","""5.0""","""Great Read as …","""A3LW89A82KHPM1…","""Great Read - F…","""false""",2013-12-04 00:00:00


In [49]:
metadata.sample(1)

asin,brand,price,title,cant_image,cant_category,rank_in_category,also_buy_count,also_view_count
str,str,f64,str,u32,u32,i64,u32,u32
"""1101946903""","""Visit Amazon's…",14.21,"""Brave Enough""",0,3,5946,50,55


Join 2 dataframes by the ASIN number

In [50]:
join_keys = ["asin"]
df_joined = metadata.join(reviews, join_keys)

In [51]:
df_joined.shape

(2618468, 15)

In [52]:
df_joined.sample(1)

asin,brand,price,title,cant_image,cant_category,rank_in_category,also_buy_count,also_view_count,overall,reviewText,reviewerID,summary,verified,unixReviewTime
str,str,f64,str,u32,u32,i64,u32,u32,str,str,str,str,str,datetime[μs]
"""045146981X""","""Visit Amazon's…",5.39,"""Llama Llama I …",0,3,3503,96,60,"""5.0""","""Sweet book for…","""A1I7CMF07WQ7FN…","""Five Stars""","""true""",2017-02-20 00:00:00


In [53]:
final = df_joined.select("title", "reviewerID", "unixReviewTime")


In [54]:
final.sample(1)

title,reviewerID,unixReviewTime
str,str,datetime[μs]
"""The Martian""","""A20GRCRYT04TJ4…",2015-08-10 00:00:00


In [55]:
final.shape
#2916355

(2618468, 3)

In [56]:
final = final.unique()

In [57]:
final.shape

(2612701, 3)

Save the file to be used

In [58]:
final.write_parquet("../data/processed/Final_data_for_ML.parquet")

In [70]:
def converTime(x):
    print(type(x))
    return time.mktime(x.timetuple()) + x.microsecond/1e6

In [71]:
final = final.with_columns(
    pl.col("unixReviewTime").apply(lambda x: converTime(x) ).alias("reviewtime")
).drop("unixReviewTime")    

PanicException: assertion failed: `(left == right)`
  left: `9`,
 right: `1`: impl error

PanicException: Unwrapped panic from Python code